In [18]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt

from src.models import ResNet50
from src.utils.common import UnCallBack
from src.utils.evaluate import makeEval
from src.utils.data_loaders import train_val_dataloader, test_dataloader, train_dataloader
%matplotlib inline

In [19]:
def commonArguments( test_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = ResNet50()
    loss_fn = nn.CrossEntropyLoss(reduction='none')
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3 )

    eval_callback = makeEval(test_loader, loss_fn, device)

    return {"model":model, "optimizer":optimizer, "loss_fn":loss_fn, "eval":eval_callback, "device":device}


In [20]:
%load_ext autoreload
%aimport src.utils.common
%aimport src.utils.evaluate
%aimport src.train_schaul
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Upper-bound and/or Loss

To use Upper-bound method use use_loss_estimation = False, \
for Loss use use_loss_estimation = True

In [21]:
from src.train_importance_sampling import  train_full_upper_bound

def upperBoundOrLoss(train_loader, test_loader, n_epochs = 50, use_loss_estimation = False):

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    model = ResNet50()
    model.to(device)

    loss_fn = nn.CrossEntropyLoss(reduction='none')
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3 )

    callback = UnCallBack( info_list = ['train_loss', 'train_acc', 'train_w_loss', 'val_loss', 'val_acc', 'train_uniform_cnt'])

    eval_callback = makeEval(test_loader, loss_fn, device)

    train_full_upper_bound( model, 
                train_loader, 
                loss_fn, 
                optimizer, 
                n_epochs = n_epochs, 
                eval = eval_callback, 
                callback = callback, 
                presample = 3, 
                tau_th = None,
                use_loss_estimation = use_loss_estimation,
                second_approach = True,
                device= device)

    if use_loss_estimation:
        callback.save("callbacks/loss")
    else:
        callback.save("callbacks/upper_bound")

    return callback



In [ ]:
from src.train_rho_loss import train_full_rho_loss
def rhoLoss(train_loader, test_loader, n_epochs = 50, train_irr_loader = None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = ResNet50()
    model.to(device)

    model_irr = ResNet50()
    model_irr.to(device)


    loss_fn = nn.CrossEntropyLoss(reduction='none')
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3 )
    optimizer_irr = torch.optim.Adam(model_irr.parameters(), lr = 1e-3 )

    callback = UnCallBack( info_list = ['train_loss', 'train_acc', 'train_w_loss', 'val_loss', 'val_acc', 'train_uniform_cnt'])

    eval_callback = makeEval(test_loader, loss_fn, device)

    train_full_rho_loss(model, 
            model_irr,
            train_loader, 
            train_irr_loader,
            loss_fn, 
            optimizer, 
            optimizer_irr,
            n_epochs=n_epochs, 
            eval=eval_callback, 
            callback=callback, 
            presample=3, 
            tau_th = None)

    callback.save("rho_loss")

# Schaul training 

In [22]:
%autoreload 1
from src.train_schaul import train_full_schaul, train_full_schaul2


def schaul(train_loader, test_loader, n_epochs = 50):


    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = ResNet50()
    model.to(device)

    loss_fn = nn.CrossEntropyLoss(reduction='none')
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3 )

    callback = UnCallBack( info_list = ['train_loss', 'train_acc', 'val_loss', 'val_acc'])


    eval_callback = makeEval(test_loader, loss_fn, device)

    train_full_schaul(model, 
                    train_loader, 
                    loss_fn, 
                    optimizer, 
                    n_epochs = n_epochs, 
                    eval = eval_callback, 
                    callback = callback, 
                    device = device)

    callback.save("callbacks/schaul")
    return callback

# Loshchilov training 

In [23]:
from src.train_loshchilov import train_full_loshchilov

def loshchilov(train_loader, test_loader, n_epochs = 50):

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = ResNet50()
    model.to(device)

    loss_fn = nn.CrossEntropyLoss(reduction='none')
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3 )

    callback = UnCallBack( info_list = ['train_loss', 'train_acc', 'val_loss', 'val_acc'])

    eval_callback = makeEval(test_loader, loss_fn, device)

    train_full_loshchilov(model, 
                        train_loader, 
                        loss_fn, 
                        optimizer, 
                        n_epochs=n_epochs, 
                        eval = eval_callback, 
                        callback=callback, 
                        device = device)

    callback.save("callbacks/loshchilov")

    return callback

# Run 

In [25]:
%autoreload 1
callback = UnCallBack( info_list = ['train_loss', 'train_acc', 'val_loss', 'val_acc'])
callback.save("callbacks/loshchilov")

The new directory is created!


In [2]:
%autoreload 1
n_epochs = 2


train_loader = train_dataloader(batch_size=300)
test_loader = test_dataloader(batch_size=120)

callback_upper_bound = upperBoundOrLoss(train_loader, test_loader, n_epochs)
callback_loss = upperBoundOrLoss(train_loader, test_loader, n_epochs, use_loss_estimation=True)



train_irr_loader  = train_dataloader(batch_size=120, subset=0.25)#128*3
train_loader, test_loader = train_val_dataloader(batch_size=120, index=True)
test_loader = test_dataloader(batch_size=120)


#train_loader = train_dataloader(batch_size=80)
#test_loader = test_dataloader(batch_size=80)
train_loader, test_loader = train_val_dataloader(batch_size=120)
schaul_callback = schaul(train_loader, test_loader, n_epochs)


train_loader = train_dataloader(batch_size=64)
test_loader = test_dataloader(batch_size=64)
loshchilov_callback = loshchilov(train_loader, test_loader, n_epochs)

NameError: name 'train_dataloader' is not defined